In [1]:
import findspark
import os
import pyspark

from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from pyspark.sql.functions import col, from_json

In [2]:
findspark.init()

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1 pyspark-shell'

In [4]:
spark = SparkSession.builder.appName('MachineMonitor').getOrCreate()

:: loading settings :: url = jar:file:/home/caio/Projetos/Kafka/.env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/caio/.ivy2/cache
The jars for the packages stored in: /home/caio/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bd779f6e-89f6-4312-a52e-d9c3b84a4f31;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resoluti

23/02/07 12:50:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/07 12:50:01 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/02/07 12:50:01 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/02/07 12:50:01 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/02/07 12:50:01 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/02/07 12:50:01 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/02/07 12:50:01 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/02/07 12:50:01 WARN Utils: Service 'sparkDriver' could not bi

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Não foi possível acessar o endereço requisitado: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:555)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.netBind(ServerSocketChannelImpl.java:344)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:301)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:134)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractCh

In [21]:
pyspark.SparkContext().setLogLevel('ERROR')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=MachineMonitor, master=local[*]) created by getOrCreate at /tmp/ipykernel_70280/1689488364.py:1 

In [5]:
df = spark \
    .readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', 'localhost:9092') \
    .option('subscribe', 'sensors-data') \
    .load()

In [6]:
sensor_data = \
    StructType([
        StructField('temperature', DoubleType(), True),
        StructField('rpm', IntegerType(), True)
    ])

In [7]:
message_schema = \
    StructType([ 
        StructField('timestamp', StringType(), True), 
        StructField('id_machine', StringType(), True), 
        StructField('sensor-data', sensor_data, True)
    ])

In [8]:
df = df.selectExpr('CAST(value AS STRING)')

In [9]:
df = df.withColumn('jsonData', from_json(col('value'), message_schema)).select('jsonData.*')

In [10]:
df.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- id_machine: string (nullable = true)
 |-- sensor-data: struct (nullable = true)
 |    |-- temperature: double (nullable = true)
 |    |-- rpm: integer (nullable = true)



## Análise de Dados em Tempo Real

In [ ]:
# Aqui temos o objeto que irá conter nossa análise, o cálculo da média das temperaturas por sensor
df_media_temp_sensor = df_conversao_temp_sensor.groupby("sensor").mean("temperatura")

In [ ]:
df_media_temp_sensor.printSchema()

In [ ]:
# Renomeamos as colunas para simplificar nossa análise
df_media_temp_sensor = df_media_temp_sensor.select(col("sensor").alias("sensor"), 
                                                   col("avg(temperatura)").alias("media_temp"))

In [ ]:
df_media_temp_sensor.printSchema()

Abaixo abrimos o streaming para análise de dados em tempo real, imprimindo o resultado no console.

In [ ]:
# Objeto que inicia a consulta ao streaming com formato de console
query = df_media_temp_sensor.writeStream.outputMode("complete").format("console").start()

Envie novos arquivos para o Kafka a fim de ver a análise em tempo real por aqui. Clique no botão Stop no menu superior para interromper a célula a qualquer momento.

In [ ]:
# Executamos a query do streaming e evitamos que o processo seja encerrado
query.awaitTermination()

In [ ]:
query.status

In [ ]:
query.lastProgress

In [ ]:
query.explain()

## Análise de Dados em Tempo Real

In [ ]:
# Objeto que inicia a consulta ao streaming com formato de memória (cria tabela temporária)
query_memoria = df_media_temp_sensor \
    .writeStream \
    .queryName("dsa") \
    .outputMode("complete") \
    .format("memory") \
    .start()

In [ ]:
# Streams ativados
spark.streams.active

In [ ]:
# Vamos manter a query executando por algum tempo e aplicando SQL aos dados em tempo real
from time import sleep

for x in range(10):
    
    spark.sql("select sensor, round(media_temp, 2) as media from dsa where media_temp > 65").show()
    sleep(3)
    
query_memoria.stop()

# Fim